**We have 8-day data from AIRS.**

In [ ]:
import datetime as dt
import os

In [ ]:
DATA_PATH='' #directory where AIRS2ST8 has been downloaded/stored."
OUTPUT_PATH="data/"

We have in total 123 granules. So each granules belongs to 8-day data from 01-01-2019 to 30-09-21. We have 123 granule names stored in granules.

In [ ]:
granules=os.listdir(DATA_PATH)
print("granules =",len(granules))

granules = 123


## Mapping criteria<br>
We have to map each granule to weeks. Mapped into 4 weeks for each month. Here week represents 8-days. Which granule will map to which week is decided on the basis of number of days overlap. Minimum 4 days should be overlapped to map to a week. Once a granule is used it can not be used again.<br>
week-1: dates 1 to 8<br>
week-2: dates 9 to 16<br>
week-3: dates 17 to 24<br>
week-4: dates 25 to end.<br>

In [ ]:
week_length=8 #days
n_weeks_per_month=4
min_overlap=4 #days

## Example File Names
8-Day Product Dec 3-10, 2009 processed using only AIRS radiances:<br>
Name: AIRS.2009.12.03.L3.RetStd_IR008.v6.0.9.0.G2002123120634.hdf<br><br>
### Making a dictionary with key as granule name and value as 8 dates that belong to that granule<br>
Year, month and day is extracted from the above name as 2009, 12, 3.<br>
Using datetime.date(2009,12,3) we will a date as 2009-12-03. These dates are stored are in dictionary (named granules_dates) as values for their granule names as keys.<br><br>
For each in key(granule) in dictionary only one value is present(date extracted from granule name). So, continously next  7 dates are added in the values. So, for 2009-12-03 next 7 dates will be (2009-12-04, 2009-12-05, 2009-12-06, 2009-12-07, 2009-12-08, 2009-12-09, 2009-12-10). Hence, every key(granule name) will have list of 8 dates as value.




In [ ]:
granule_dates={}
for granule in granules:
    values=granule.split(".")
    year=int(values[1])
    month=int(values[2])
    day=int(values[3])
    date=dt.date(year,month,day)
    granule_dates[granule]=[date]
    for i in range(7):
        date=date+dt.timedelta(days=1)
        granule_dates[granule].append(date)

## Making a dictionary with key as week and value as the dates falling that week<br>
For loop over 3 years namely 2019, 2020, 2021, over 12 months i.e. 1 to 12 and over 4 weeks (weeks starting from 1) and day starting from 1st. (i.e. starting from 2019-01-01).<br><br>Making a dictionary (named week_dates) with keys as start of week and values as the days falling in that week. Fetching date after every 8 days for 4 times(4 weeks in a month) (say 2019-01-01, 2019-01-09, 2019-01-17, 2019-01-25).<br><br>The innermost loop is running from 0 to 7 as we have found 1 date in the last date. So, for every date extracted in the above step we are appending consecutive 7 more dates. The only one more constraint is all the dates should be of same month, as soon as month changes the innermost loop breaks, giving dates of same month only.<br><br> Finally the dictionary is made as say: Key = 1.2019.week1 and its value is [2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05, 2019-01-06, 2019-01-07, 2019-01-08]

In [ ]:
week_dates={}
months=[i+1 for i in range(12)]
years=[2019,2020,2021]
for year in years:
    for month in months:
        for week in range(1,n_weeks_per_month+1,1):
            date=dt.date(year,month,1)+dt.timedelta(days=week_length*(week-1))
            dates=[date]
            for i in range(week_length-1):
                new_date=date+dt.timedelta(days=1)
                if new_date.month==date.month:
                    dates.append(new_date)
                    date=new_date
                else:
                    break
            week_dates[str(month)+"."+str(year)+"."+"week"+str(week)]=dates                

## Making a dictionary 
Now we have week_dates dictionary with keys as name of week and values as the days falling in that week.<br>
Another dictionary granule_dates with key as granule name and value as 8 dates that belong to that granule.<br><br>

Now, Take intersection of dates stored in week_dates for evey week with dates stored in granule_dates for every granule. If minimum over lap is equal or more than 4 then put the total number of intersected dates as value in intersection dictionary with key as granule name.<br><br>
If the length of intersection dictionary is equal or greater than 1 then put week as key in week_granule dictionary with value as max of zip of intersection values and keys. By this we will get the key (granule name) which has max number of intersection.<br><br> We want every granule to be used as once only hence the for loop is over unused granules using set intersection.



In [ ]:
week_granule={}
for week in week_dates:
    intersection={}
    for granule in set(granules).difference(set(week_granule.values())):
        value=len(set(week_dates[week]).intersection(granule_dates[granule]))
        if value>=min_overlap:
            intersection[granule]=value
    if len(intersection)>=1:
        week_granule[week]=max(zip(intersection.values(), intersection.keys()))[1]

## Making a csv file of week mappings
File name week_mappings_airs.csv with columns week and granule, showing the mapping of which granule falls in which week. Use week_granule dictionary from above step which has key as week, store it in week column and its value (i.e. name of granule) under granule column.

In [ ]:
fp=open(OUTPUT_PATH+"week_mappings_airs.csv","w")
fp.write("week,granule\n")
for week in week_granule:
    fp.write(week+","+week_granule[week]+"\n")
fp.close()